<a target="_blank" href="https://colab.research.google.com/github/FlorianWolff95/DHBW_BI_S24/blob/main/ML_Fallstudie_Obesity.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# ML Fallstudie Obesity
Sie finden in unserem [Repo](https://github.com/AlexKressner/Business_Intelligence) unter `Daten/Obesity/` Daten zur Schätzung der Adipositaslevel von Personen aus den Ländern Mexiko, Peru und Kolumbien, im Alter zwischen 14 und 61 Jahren, mit unterschiedlichen Essgewohnheiten und körperlicher Verfassung. Die Daten wurden mit Hilfe einer Webplattform gesammelt, auf der anonyme Nutzer jede Frage eines Fragebogens beantworteten. Anschließend wurden die Informationen verarbeitet, wobei 17 Attribute und 2111 Datensätze erhalten wurden.

Die mit den Essgewohnheiten verbundenen Attribute sind: Häufiger Konsum von hochkalorischen Lebensmitteln (FAVC), Häufigkeit des täglichen Gemüsekonsums (FCVC), Anzahl der Hauptmahlzeiten pro Tag (NCP), Konsum von Nahrungsmitteln zwischen den Mahlzeiten (CAEC), täglicher Wasserkonsum (CH20) und Alkoholkonsum (CALC).

Die mit der körperlichen Verfassung verbundenen Attribute sind: Überwachung des Kalorienverbrauchs (SCC), Häufigkeit körperlicher Aktivitäten (FAF), Zeit, die mit Technologiegeräten verbracht wird (TUE), genutztes Verkehrsmittel (MTRANS) und die erhaltenen Variablen: Geschlecht, Alter, Größe und Gewicht.

Die verschiedenen Adipositatswerte (in der Spalte `ObesityLevel`) sind:

- Untergewicht
- Normalgewicht
- Übergewicht_I
- Übergewicht_II
- Adipositas_I
- Adipositas_II
- Adipositas_III

Die Daten stammen von [Kaggle](https://www.kaggle.com/datasets/aravindpcoder/obesity-or-cvd-risk-classifyregressorcluster). Ihre Aufgabe besteht darin, ein geeignetes Modell zur Vorhersage des Adipositatlevels basierend auf den gegebenen Features zu erstellen. Beantworten Sie dafür die folgenden Fragestellungen bzw. gehen Sie wie folgt vor:

1. **Vorüberlegung**: Wenn Sie das Adipositaslevel einer Person vorhersagen, handelt es sich um eine Regression oder Klassifikation?

2. **Datenexploration**:
- Wie viele Features gibt es?
- Gibt es fehlende Werte bei den Features?
- Wie sind die numerischen Features verteilt, d.h. berechnen Sie Kenngrößen wie den Mittelwert, Median, Quartile, etc.? Hinweis: Dafür gibt es eine Funktion!
- Wie ist die Korrelation zwischen dem Target und den Features? Hinweis: Dafür gibt es eine Funktion!

3. **Feature Engineering**:
- Achte Sie auf die Datentypen beim Erstellen von Features!
- Gibt es aus Ihrer Sicht weitere interessante Features, die Sie aus den Daten ableiten können?

4. **Modell trainieren und bewerten**:
- Entwickeln Sie ein Modell, mit dem Sie das Adipositaslevel einer Person basierend auf den zur Verfügung stehenden Features schätzen können.
- Optimieren Sie die Hyperparamter Ihres Modells.
- Können Sie Prognosegüte erhöhen, wenn Sie zwei Modelle trainieren? Ein Modell schätzt das Adipositaslevels für Männer, das andere für Frauen.

5. **Prognose**: Schätzen Sie das Adipositaslevel einer beliebigen Person.

In [ ]:
#! git clone https://github.com/FlorianWolff95/DHBW_BI_S24

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os


WORKING_DIR = os.getcwd()

if not os.path.exists(os.path.join(WORKING_DIR, "Daten")):
    DATA_DIR = os.path.join(WORKING_DIR, "DHBW_BI_S24", "Daten")
else:
    DATA_DIR = os.path.join(WORKING_DIR, "Daten")

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR, "Obesity", "ObesityDataSet.csv"))
df.columns = df.columns.str.strip().str.lower()
df.head()

In [ ]:
full_col_names = dict(
    {
        "fcvc": "Frequent consumption of high caloric food",
        "fcva": "Frequency of consumption of vegetables",
        "ncp": "Number of main meal",
        "caec": "Consumption of food between meals",
        "ch2o": "Consumption of water daily",
        "scc": "Calories consumption monitoring",
        "faf": "Physical activity frequency",
        "tue": "Time using technology devices",
        "calc": "Consumption of alcohol",
        "mtrans": "Transportation used",
    }
)

## 1. **Vorüberlegung:** Es handelt sich um MultiClass Klassifikation.

## 2. **Datenexploration**

Es ist das Ziel anhand des Essverhaltens, Alter und Geschlecht vorherzusagen welches Adiporitaslevel `NObeyesdad` hat.
Da `NObeyesdad` bestimmt wird über den `BMI`, welcher sich berechnet aus `height` und `weight`, können diese Spalten nicht zur Vorhersage verwendet werden.

In [ ]:
FEATURES = [
    "age",
    "family_history_with_overweight",
    "favc",
    "ncp",
    "caec",
    "smoke",
    "ch2o",
    "scc",
    "faf",
    "tue",
    "mtrans",
    "calc",
]
TARGET = "nobeyesdad"

In [ ]:
df.info()

In [ ]:
col_cat = ["gender", "caec", "calc", "mtrans", "nobeyesdad"]
cat_bool = ["family_history_with_overweight", "favc", "smoke", "scc"]
cat_num = df.select_dtypes("float64").columns.tolist()

df[col_cat] = df[col_cat].astype("category")

df[cat_bool] = df[cat_bool].replace({"no": 0, "yes": 1}).astype("bool")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Check for missing values
df.isna().sum()

In [ ]:
# distributions of col_cat
for x in col_cat:
    print(df[x].value_counts(normalize=True))
    print("\n")

In [ ]:
fig, ax = plt.subplots(len(cat_num), 1, figsize=(15, 30), sharex=False)
sns.set_style("whitegrid")
for i, col in enumerate(cat_num):
    sns.violinplot(x=TARGET, y=col, data=df, ax=ax[i], hue="gender", split=False)
    if col in full_col_names.keys():
        ax[i].set_ylabel(full_col_names[col])

plt.tight_layout()
plt.show()

In [ ]:
col_features_cat = df[FEATURES].select_dtypes("category").columns.tolist() + cat_bool
_, ax = plt.subplots(
    len(col_features_cat),
    2,
    figsize=(15, len(col_features_cat) * 4),
    width_ratios=[1, 4],
)
sns.set_theme(style="whitegrid")
for i, col in enumerate(col_features_cat):
    sns.countplot(y=col, data=df, ax=ax[i, 0])
    sns.countplot(x=col, data=df, ax=ax[i, 1], hue=TARGET)
    if col in full_col_names.keys():
        ax[i, 0].set_ylabel(full_col_names[col])
        ax[i, 1].set_xlabel(full_col_names[col])

plt.tight_layout()
plt.show()

## 4. Modell trainieren

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    GridSearchCV,
)
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

import joblib


if not os.path.exists(os.path.join(WORKING_DIR, "models")):
    MODEL_DIR = os.path.join(WORKING_DIR, "DHBW_BI_S24", "models")
else:
    MODEL_DIR = os.path.join(WORKING_DIR, "models")

model_name = "xgb_model_obesity"
model_path = os.path.join(MODEL_DIR, model_name + ".joblib")

In [ ]:
df[cat_num] = df[cat_num].round(0).astype("int")

one_hot_enc = ColumnTransformer(
    transformers=[
        (
            "cat",
            OneHotEncoder(),
            df[FEATURES].select_dtypes("category").columns.tolist(),
        ),
    ],
    remainder="passthrough",
)

label_enc = LabelEncoder()
X = one_hot_enc.fit_transform(df[FEATURES])
y = label_enc.fit_transform(df[TARGET])


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
param_grid = {
    "n_estimators": [100, 500, 1000, 2000],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.001, 0.01, 0.1],
    "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.4, 0.6, 0.8, 1.0],
    "gamma": [0, 0.25, 0.5, 1.0],
    "min_child_weight": [1, 5, 10, 15, 20],
}

classifier = XGBClassifier(
    n_jobs=-1,
    random_state=42,
    booster="dart",
    tree_method="hist",
    verbosity=2,
    num_parallel_tree=4,
    objective="multi:softmax",
    eval_metric="mlogloss",
    num_class=7,
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring="accuracy",
    cv=cv,
    verbose=2,
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

joblib.dump(grid_search.best_estimator_, model_path)

In [ ]:
xgb_model = joblib.load(model_path)
y_pred = xgb_model.predict(X_test)

y_test_inv = label_enc.inverse_transform(y_test)
y_pred_inv = label_enc.inverse_transform(y_pred)

cm = confusion_matrix(y_test_inv, y_pred_inv)
plt.figure(figsize=(10, 10))
sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=label_enc.classes_,
    yticklabels=label_enc.classes_,
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

print("-" * 100 + "\n" + f"Accuracy: {accuracy_score(y_test_inv, y_pred_inv)}\n")
print(
    f'f1-score (combines precision and recall): {f1_score(y_test_inv, y_pred_inv, average="weighted")}\n'
)
print(
    f'Precision (accuracy of positve -> high cost false positives): {precision_score(y_test_inv, y_pred_inv, average="weighted")}\n'
)
print(
    f'Recall (Sensitivity to capture all positive instances -> high cost missing positives): {recall_score(y_test_inv, y_pred_inv, average="weighted")}\n'
    + "-" * 100
    + "\n"
)